In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import pyproj

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# GeoPandas


<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[CRS](#section_crs)

[Análisis de las proyecciones](#section_analisis)

[Reproyección](#section_reproyeccion)

---



## Proyección


<a id="section_intro"></a> 
###  Intro
[volver a TOC](#section_toc)

Como vimos anteriormente, cada figura geométrica en Geopandas representa un objeto que requiere de un *sistema de coordenadas* que lo relacione con un lugar específico de la Tierra. Si bien el sistema más comun es el que usa latitud y longitud para indicar la posición del objeto, existen muchos otros.

Las **proyecciones** intentan representar la superficie de la tierra o una porción de la tierra en una hoja plana de papel o en la pantalla de una computadora. Luego, un *sistema de referencia de coordenadas (CRS)* define, con la ayuda de coordenadas, cómo se relaciona ese mapa *bidimensional* proyectado con lugares reales de la tierra. 

En otras palabras, el **Sistema de Referencia de Coordenadas (Coordinate Reference System en inglés) (CRS)** le indica a Python el lugar geográfico real donde se encuentra los objetos representados por las formas geométricas.

Por otra parte, es importante que todos los objetos se expresen con el mismo CRS, para que las operaciones entre ellos se pueda realizar. Como algunas veces no ocurre eso, tenemos que realizar una **reproyección** de algunos objetos, es decir, cambiarlos de sistema de coordenadas, para unificar el CRS de todos los objetos.

Se utilizan diferentes proyecciones cartográficas según el tipo de mapa que se desea crear, ya que existen determinadas proyecciones que se adaptan mejor a unos usos concretos que a otros, a unas zonas u otras. Por ejemplo, una proyección que representa con exactitud la forma de los continentes distorsiona, por el contrario, sus tamaños relativos. Hay que recordar que toda proyección es una representación de la realidad.

<a id="section_crs"></a> 
###  CRS
[volver a TOC](#section_toc)

Existen numerosos CRS, según las variadas necesidades de geolocalización.

Los CRS se identifican por un código llamado **EPSG**. Por ejemplo, el sistema de coordenadas que usa latitud y longitud es el CRS WGS 84, el cual se representa con el EPSG 4326.

Para ver la lista actualizada de EPSG ir a https://epsg.org/home.html 

Ejemplos de CRS:

- WGS 84. 

Usa la latitud y la longitud. La explicación se encuentra en la notebook "1_datos_geoespaciales".

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_1_008_Latitud_Longitud.PNG' alt="Latitud y Longitud" width=50% height=40%>
        <p><i>Latitud y Longitud</i></pEsfera terrestre dividida en franjas de 6 grados
    </div>
</div> 
      
- UTM. Universal Transverse Mercator.
    
Se representa con un mapa rectangular. Quizá recordemos los mapas que usabamos en la escuela secundaria para representar el planeta.
Esta proyección "aplana" la esfera terrestre, y la divide en 60 zonas de norte a sur, cada una de 6 grados.
Luego, para identificar un lugar, se necesita un número para la ubicación norte-sur, otro para la posición este-oeste (ambos positivos), y la zona a la que pertenece.
    
Es util para mapear areas pequeñas.

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_4_001_UTM.PNG' alt="Esfera terrestre dividida en franjas de 6 grados" width=25% height=40%>
        <p><i>Esfera terrestre dividida en franjas de 6 grados</i></p>
    </div>
</div>

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_4_002_UTM.PNG' alt="Mapa rectangular" width=25% height=40%>
        <p><i>Mapa rectangular</i></p>
    </div>
</div>

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_4_003_UTM.PNG' alt="Coordenadas de Lima, con CRS WGS 84 y UTM" width=35% height=80%>
        <p><i>Coordenadas de Lima, con CRS WGS 84 y UTM</i></p>
        <p><i><a href="https://www.latlong.net/lat-long-utm.html" target="_blank">https://www.latlong.net/lat-long-utm.html</a></i></p>        
    </div>
</div>

<a id="section_analisis"></a> 
###  Análisis de las proyecciones
[volver a TOC](#section_toc)

Cuando usamos Geopandas, su estructuras de datos GeoSeries y GeoDataFrame deben contener al menos una columna que indique, mediante una forma geométrica, su posición geoespacial.

Pero en muchos archivos disponibles, los datos geoespaciales de los objetos se expresan con dos campos numéricos. Uno para la latitud y otro para la longitud.

In [2]:
df_comisarias = pd.read_csv("../Data/comisarias.csv")
df_comisarias.head()

,X,Y,NOMBRE
0,-58.468944,-34.683121,COMISARIA 52
1,-58.474649,-34.679169,COMISARIA 48
2,-58.501166,-34.661994,COMISARIA 42
3,-58.431981,-34.660395,COMISARIA 36
4,-58.402830,-34.641950,COMISARIA COMUNA 4


Primero debemos transformar los datos numéricos en una nueva columna que identifique a una forma geométrica. Recordemos que la columna se llama por default *geometry*.

En este caso, se transforma la localización de las comisarias en un *punto*.

In [3]:
geo_comisarias = gpd.GeoDataFrame(df_comisarias
                , geometry=gpd.points_from_xy(df_comisarias.X, df_comisarias.Y))
geo_comisarias.head()

,X,Y,NOMBRE,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-58.46894 -34.68312)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-58.47465 -34.67917)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-58.50117 -34.66199)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-58.43198 -34.66039)
4,-58.402830,-34.641950,COMISARIA COMUNA 4,POINT (-58.40283 -34.64195)


Cuando buscamos el CRS usado en el punto, no aparece nada. Porque solo transformamos los numeros X e Y, en las coordenadas de la figura.

Tenemos que indicar cual es el CRS explicitamente, con el método *crs*

In [4]:
geo_comisarias.crs

In [5]:
geo_comisarias.crs = {'init' :'epsg:4326'}

/home/pablo/anaconda3/envs/dhdsblend2021_geopandas/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
 geo_comisarias.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

<a id="section_reproyeccion"></a> 
###  Reproyección
[volver a TOC](#section_toc)

A veces, necesitamos cambiar el CRS que estamos usando, para transformar las coordenadas.

Vamos a llevar a las coordenadas que disponen las comisarias, WGS 84, por latitud y longitud (EPSG=4326), a unas coordenadas por UTM (EPSG=32733).

In [7]:
 geo_comisarias.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
geo_comisarias.head()

,X,Y,NOMBRE,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-58.46894 -34.68312)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-58.47465 -34.67917)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-58.50117 -34.66199)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-58.43198 -34.66039)
4,-58.402830,-34.641950,COMISARIA COMUNA 4,POINT (-58.40283 -34.64195)


In [9]:
geo_comisarias = geo_comisarias.to_crs(epsg = 32733)
# este comando tira un warning. Es un problema de Geopandas al usar la libreria pyproj.

In [10]:
geo_comisarias.head()

,X,Y,NOMBRE,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-6304210.347 2483981.461)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-6305231.264 2483555.406)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-6309786.922 2481474.256)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-6305310.907 2490738.920)
4,-58.402830,-34.641950,COMISARIA COMUNA 4,POINT (-6306256.037 2496114.174)


In [11]:
geo_comisarias.crs

<Projected CRS: EPSG:32733>
Name: WGS 84 / UTM zone 33S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, southern hemisphere between 80°S and equator, onshore and offshore. Angola. Congo. Democratic Republic of the Congo (Zaire). Gabon. Namibia. South Africa.
- bounds: (12.0, -80.0, 18.0, 0.0)
Coordinate Operation:
- name: UTM zone 33S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

---

### Referencias

Tipos de CRS

https://epsg.org/home.html

CRS - Coordinate Reference Systems

https://docs.qgis.org/2.8/en/docs/gentle_gis_introduction/coordinate_reference_systems.html

UTM - Universal Trasverse Mercator

https://gisgeography.com/utm-universal-transverse-mercator-projection/